In [60]:
# !pip install --upgrade pip
# !pip install xgboost 

In [30]:
import os 
import random
import pandas as pd
import numpy as np
from copy import deepcopy
import pickle
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

In [2]:
def set_random_seed(seed):
    np.random.seed(seed)
    random.seed(seed)

In [17]:
seed = 1
set_random_seed(seed)

base_dir = './dataset/'

# load the positive and negative QA pairs
all_pos_df = pd.read_csv(os.path.join(base_dir, 'positive_general_labels.csv'))
all_neg_df = pd.read_csv(os.path.join(base_dir, 'negative_labels.csv'))

# shuffle each set individually
# p = np.random.permutation(all_pos_df.index)
# all_pos_df = all_pos_df.reindex(p)
# all_neg_df = all_neg_df.reindex(p)

print('All positive samples:', all_pos_df.shape)
print('All negative samples:', all_neg_df.shape)

# check a sample data point for positive and negative labels
print(all_pos_df.iloc[10])
print(all_neg_df.iloc[10])


# put the first "test_size" QA pos and neg labels in the test dataset
test_size = 500
test_df = all_pos_df[:test_size]
test_df = test_df.append(all_neg_df[:test_size])

# put the second "test_size" QA pos and neg labels in the validation dataset
valid_df = all_pos_df[test_size:2*test_size]
valid_df = valid_df.append(all_neg_df[test_size:2*test_size])

# put all the other pairs in the training dataset
train_df = all_pos_df[2*test_size:]
train_df = train_df.append(all_neg_df[2*test_size:])

print('Training dataset:', train_df.shape)
print('Validation dataset:', valid_df.shape)
print('Test dataset:', test_df.shape)

All positive samples: (4410, 5)
All negative samples: (4137, 5)
question_id                             56bc751eac7ad10019000013
question          Name synonym of Acrokeratosis paraneoplastica.
answer         Acrokeratosis paraneoplastic (Bazex syndrome) ...
label                                                       True
source                                     BioASQ_TR_IdealAnswer
Name: 10, dtype: object
question_id                             56bc751eac7ad10019000013
question          Name synonym of Acrokeratosis paraneoplastica.
answer         he conditioned infections due to opportunistic...
label                                                      False
source                                                   CORD-19
Name: 10, dtype: object
Training dataset: (6547, 5)
Validation dataset: (1000, 5)
Test dataset: (1000, 5)


In [18]:
# verify that the questions in different sets do NOT overlap
test_qs = set(test_df['question'].values)
valid_qs = set(valid_df['question'].values)
train_qs = set(train_df['question'].values)
print('test', len(test_qs), ' - train', len(train_qs), ' - intersection:', test_qs.intersection(train_qs))
print('test', len(test_qs), ' - valid', len(valid_qs), ' - intersection:', test_qs.intersection(valid_qs))
print('valid', len(valid_qs), ' - train', len(train_qs), ' - intersection:', valid_qs.intersection(train_qs))

test 500  - train 3410  - intersection: set()
test 500  - valid 500  - intersection: set()
valid 500  - train 3410  - intersection: set()


In [19]:
def clustering_active_learner(data, b_size, probs):
        
    X = np.zeros((data.shape[0], len(sentences_embeds[0])))
    for i in range(data.shape[0]):
        s_id = sentence_ids[data.iloc[i]['question'] + ' ||| ' + data.iloc[i]['answer']]
        X[i, :] = sentences_embeds[s_id]
        
    kernel = KMeans(n_clusters=b_size, verbose=0, n_jobs=-1, random_state=1) 
    kernel = kernel.fit(X)     
    labels = kernel.predict(X)

    cluster_min = np.ones((b_size, 2))
    sampled = pd.DataFrame(columns=data.columns)
    for i in range(data.shape[0]):
        if np.max(probs[i, :]) < cluster_min[labels[i], 0]:
            cluster_min[labels[i], 0] = np.max(probs[i, :])
            cluster_min[labels[i], 1] = i

    for i in cluster_min[:, 1]:
        sampled = sampled.append(data.iloc[int(i)])

    remaining = data.drop(sampled.index, axis=0)

    return sampled, remaining
    

In [20]:
def uncertainty_active_learner(data, b_size, probs):
    
    # step 1: find the higher probability
    # step 2: sort these probabilities
    # step 3: choose the ones that have the lowest probability
    sorted_arg = np.argsort(np.amax(probs, axis=1))
    
    sampled = data.drop(list(sorted_arg[b_size:]))
    remaining = data.drop(list(sorted_arg[:b_size]))

    return sampled, remaining

In [21]:
def random_active_learner(data, b_size=10, seed=0):
    sampled = data.sample(n=b_size, random_state=seed)
    remaining = data.drop(sampled.index)

    return sampled, remaining
    

In [22]:
sentence_ids = pickle.load(open('pos_general_neg_bert_sentence_ids.pkl', 'rb'))
sentences_embeds = pickle.load(open('pos_general_neg_bert_sentence_embeddings.pkl', 'rb'))

def to_features(data):
    X = np.zeros((data.shape[0], 768))
    Y = np.zeros((data.shape[0], 1), dtype=bool)
    
    for i in range(data.shape[0]):
        s_id = sentence_ids[data.iloc[i]['question'] + ' ||| ' + data.iloc[i]['answer']]
        X[i, :] = sentences_embeds[s_id]
        Y[i, :] = data.iloc[i]['label']
        
    return X, Y

In [23]:
def eval_results(model, X_train, Y_train, X_valid, Y_valid, X_test, Y_test, eval_metrics):
    Y_train_pred = model.predict(X_train)
    train_predictions = [round(value) for value in Y_train_pred]
    train_accuracy = accuracy_score(Y_train, train_predictions)
    train_f1 = f1_score(Y_train, train_predictions)
    print('Train accuracy:', train_accuracy, ' - Train f1:', train_f1)

    Y_valid_pred = model.predict(X_valid)
    valid_predictions = [round(value) for value in Y_valid_pred]
    valid_accuracy = accuracy_score(Y_valid, valid_predictions)
    valid_f1 = f1_score(Y_valid, valid_predictions)
    print('Valid accuracy:', valid_accuracy, ' - Valid f1:', valid_f1)

    Y_test_pred = model.predict(X_test)
    test_predictions = [round(value) for value in Y_test_pred]
    test_accuracy = accuracy_score(Y_test, test_predictions)
    test_f1 = f1_score(Y_test, test_predictions)
    print('Test accuracy:', test_accuracy, ' - Test f1:', test_f1)

    eval_metrics.append([train_accuracy, train_f1, valid_accuracy, valid_f1, test_accuracy, test_f1])
    return test_accuracy, test_f1

In [24]:
xgb_params = {
    "n_jobs": -1, 
    "max_depth": 25, 
    "min_child_weight": 0.1,
    "gamma": 1,
    "eta": 0.001,
    "num_round": 10000,
    "subsample": 0.5,
    "colsample_bytree": 0.5,
    "verbosity": 0, 
    "tree_method": 'hist', 
    "objective": 'binary:logistic'    
}

xgb_model = XGBClassifier()
xgb_model.set_params(**xgb_params)
X_train, Y_train = to_features(train_df)
X_valid, Y_valid = to_features(valid_df)
X_test, Y_test = to_features(test_df)

print('Started fitting...')
xgb_model.fit(X_train, Y_train,
              eval_set=[(X_train, Y_train), (X_valid, Y_valid)],
              eval_metric='error',
              verbose=False)
print('Finished fitting.')
eval_metrics = []
print('Started predicting...')
acc, f1 = eval_results(xgb_model, X_train, Y_train, X_valid, Y_valid, X_test, Y_test, eval_metrics)
print('Finished predicting.')
print(acc, f1)

Started fitting...


/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Finished fitting.
Started predicting...
Train accuracy: 0.9883916297540858  - Train f1: 0.9887773183697578
Valid accuracy: 0.885  - Valid f1: 0.8841893252769387
Test accuracy: 0.904  - Test f1: 0.904
Finished predicting.
0.904 0.904


In [25]:
pickle.dump(eval_metrics, open('pos_general_neg_bert_eval_metrics-AllTraining.pkl', 'wb'))

In [26]:
seed = 0
set_random_seed(seed)

batch_size = 20

pool_df = deepcopy(train_df)
pool_df.reset_index(drop=True, inplace=True)
annotated_df = pd.DataFrame(columns=pool_df.columns)

seed_df, pool_no_seed_df = random_active_learner(data=pool_df, 
                                               b_size=batch_size, 
                                               seed=seed)

pool_no_seed_df.reset_index(drop=True, inplace=True)
seed_df.reset_index(drop=True, inplace=True)    

#X_train, Y_train = to_features(pool_df)
X_train, Y_train = to_features(seed_df)
X_valid, Y_valid = to_features(valid_df)
X_test, Y_test = to_features(test_df)
    
print(X_train.shape)
print(X_valid.shape)
print(X_test.shape)

print(np.unique(Y_train))
print(np.unique(Y_valid))
print(np.unique(Y_test))

(20, 768)
(1000, 768)
(1000, 768)
[False  True]
[False  True]
[False  True]


In [27]:
xgb_params = {
    "n_jobs": -1, 
    "max_depth": 25, 
    "min_child_weight": 0.1,
    "gamma": 1,
    "eta": 0.001,
    "num_round": 10000,
    "subsample": 0.5,
    "colsample_bytree": 0.5,
    "verbosity": 0, 
    "tree_method": 'hist', 
    "objective": 'binary:logistic'    
}

xgb_model = XGBClassifier()
xgb_model.set_params(**xgb_params)

print('Started fitting...')
xgb_model.fit(X_train, Y_train,
              eval_set=[(X_train, Y_train), (X_valid, Y_valid)],
              eval_metric='error',
              verbose=False)
print('Finished fitting.')
eval_metrics = []
pickle.dump(xgb_model, open('bert_xgboost_start_model.pkl', "wb"))
print('Started predicting...')
acc, f1 = eval_results(xgb_model, X_train, Y_train, X_valid, Y_valid, X_test, Y_test, eval_metrics)
print('Finished predicting.')
print(acc, f1)

Started fitting...


/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Finished fitting.
Started predicting...
Train accuracy: 1.0  - Train f1: 1.0
Valid accuracy: 0.651  - Valid f1: 0.5199449793672627
Test accuracy: 0.63  - Test f1: 0.48467966573816157
Finished predicting.
0.63 0.48467966573816157


In [28]:
batch_size = 5

strategy_results = {}
for al_strategy in ['uncertainty', 'random', 'clustering']:
    print("Using {} Strategy ====>".format(al_strategy))
    al_iterations = 50
    
    xgb_model = pickle.load(open('bert_xgboost_start_model.pkl', "rb"))
    
    eval_metrics = []
    print('Started predicting...')
    acc, f1 = eval_results(xgb_model, X_train, Y_train, X_valid, Y_valid, X_test, Y_test, eval_metrics)
    print('Finished predicting.')

    pool_df = deepcopy(pool_no_seed_df)
    pool_df.reset_index(drop=True, inplace=True)
    annotated_df = deepcopy(seed_df)
    print('Pool size:', pool_df.shape, ' - Annotated size:', annotated_df.shape)
    
    f1_list = [f1]
    acc_list = [acc]
    
    for i in range(al_iterations):
        print('********** AL Iteration', i, '**************')

        print('Evaluate pool dataset...')
        X_pool, Y_pool = to_features(pool_df)
        predicted_prob = xgb_model.predict_proba(X_pool)
        print('Evaluation Done.')

        if al_strategy == 'random':
            to_annotate_i, pool_df = random_active_learner(data=pool_df, 
                                                   b_size=batch_size, 
                                                   seed=seed)
        elif al_strategy == 'uncertainty':
            to_annotate_i, pool_df = uncertainty_active_learner(data=pool_df, 
                                                        b_size=batch_size, 
                                                        probs=predicted_prob)
        elif al_strategy == 'uncertainty_diversity':
            to_annotate_i, pool_df = uncertainty_diversity_active_learner(data=pool_df, 
                                                                          b_size=batch_size, 
                                                                          probs=predicted_prob)
        elif al_strategy == 'clustering':
            to_annotate_i, pool_df = clustering_active_learner(data=pool_df, 
                                                              b_size=batch_size, 
                                                              probs=predicted_prob)
        else:
            print('ERROR: Wrong AL Option.')
            break

        pool_df.reset_index(drop=True, inplace=True)
        to_annotate_i.reset_index(drop=True, inplace=True)

        annotated_df = pd.concat([annotated_df, to_annotate_i], axis=0, join='outer')
        annotated_df.reset_index(drop=True, inplace=True)

        print('Pool:', pool_df.shape, ' - to_annotate_i:', to_annotate_i.shape, ' - annotated:', annotated_df.shape)

        print('Creating features...')
        X_train, Y_train = to_features(annotated_df)
        print('Features created.')

        print('Started fitting...')
        xgb_model = XGBClassifier()
        xgb_model.set_params(**xgb_params)

        xgb_model.fit(X_train, Y_train,
                      eval_set=[(X_train, Y_train), (X_valid, Y_valid)],
                      eval_metric='logloss',
                      verbose=False)
        print('Finished fitting.')

        print('Started predicting...')
        acc, f1 = eval_results(xgb_model, X_train, Y_train, X_valid, Y_valid, X_test, Y_test, eval_metrics)
        acc_list.append(acc)
        f1_list.append(f1)
        print('Finished predicting.')
        
    strategy_results[al_strategy] = [acc_list, f1_list]
    if al_strategy == 'clustering':
        pickle.dump(eval_metrics, open('pos_general_neg_bert_eval_metrics-ALC1-bs5.pkl', 'wb'))
    elif al_strategy == 'uncertainty':
        pickle.dump(eval_metrics, open('pos_general_neg_bert_eval_metrics-ALU1-bs5.pkl', 'wb'))
    else:
        pickle.dump(eval_metrics, open('pos_general_neg_bert_eval_metrics-RND1-bs5.pkl', 'wb'))

Using uncertainty Strategy ====>
Started predicting...
Train accuracy: 1.0  - Train f1: 1.0
Valid accuracy: 0.651  - Valid f1: 0.5199449793672627
Test accuracy: 0.63  - Test f1: 0.48467966573816157
Finished predicting.
Pool size: (6527, 5)  - Annotated size: (20, 5)
********** AL Iteration 0 **************
Evaluate pool dataset...
Evaluation Done.
Pool: (6522, 5)  - to_annotate_i: (5, 5)  - annotated: (25, 5)
Creating features...
Features created.
Started fitting...


/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Finished fitting.
Started predicting...
Train accuracy: 1.0  - Train f1: 1.0
Valid accuracy: 0.704  - Valid f1: 0.6433734939759036
Test accuracy: 0.711  - Test f1: 0.6409937888198758
Finished predicting.
********** AL Iteration 1 **************
Evaluate pool dataset...
Evaluation Done.
Pool: (6517, 5)  - to_annotate_i: (5, 5)  - annotated: (30, 5)
Creating features...
Features created.
Started fitting...
Finished fitting.
Started predicting...
Train accuracy: 1.0  - Train f1: 1.0
Valid accuracy: 0.756  - Valid f1: 0.7239819004524887
Test accuracy: 0.753  - Test f1: 0.709753231492362
Finished predicting.
********** AL Iteration 2 **************
Evaluate pool dataset...
Evaluation Done.
Pool: (6512, 5)  - to_annotate_i: (5, 5)  - annotated: (35, 5)
Creating features...
Features created.
Started fitting...
Finished fitting.
Started predicting...
Train accuracy: 1.0  - Train f1: 1.0
Valid accuracy: 0.78  - Valid f1: 0.7577092511013216
Test accuracy: 0.806  - Test f1: 0.7853982300884955
Fin

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Finished fitting.
Started predicting...
Train accuracy: 1.0  - Train f1: 1.0
Valid accuracy: 0.709  - Valid f1: 0.6420664206642067
Test accuracy: 0.726  - Test f1: 0.6625615763546798
Finished predicting.
********** AL Iteration 1 **************
Evaluate pool dataset...
Evaluation Done.
Pool: (6517, 5)  - to_annotate_i: (5, 5)  - annotated: (30, 5)
Creating features...
Features created.
Started fitting...


...

Finished fitting.
Started predicting...
Train accuracy: 1.0  - Train f1: 1.0
Valid accuracy: 0.885  - Valid f1: 0.8834853090172239
Test accuracy: 0.892  - Test f1: 0.889795918367347
Finished predicting.
********** AL Iteration 48 **************
Evaluate pool dataset...
Evaluation Done.
Pool: (6282, 5)  - to_annotate_i: (5, 5)  - annotated: (265, 5)
Creating features...
Features created.
Started fitting...


/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Finished fitting.
Started predicting...
Train accuracy: 1.0  - Train f1: 1.0
Valid accuracy: 0.873  - Valid f1: 0.868937048503612
Test accuracy: 0.893  - Test f1: 0.8920282542885973
Finished predicting.
********** AL Iteration 49 **************
Evaluate pool dataset...
Evaluation Done.
Pool: (6277, 5)  - to_annotate_i: (5, 5)  - annotated: (270, 5)
Creating features...
Features created.
Started fitting...


/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Finished fitting.
Started predicting...
Train accuracy: 1.0  - Train f1: 1.0
Valid accuracy: 0.888  - Valid f1: 0.8864097363083163
Test accuracy: 0.897  - Test f1: 0.896896896896897
Finished predicting.


In [31]:
ACH_performance = np.asarray(pickle.load(open('pos_general_neg_bert_eval_metrics-AllTraining.pkl', 'rb')))
ALU_performance = np.asarray(pickle.load(open('pos_general_neg_bert_eval_metrics-ALU1-bs5.pkl', 'rb')))
ALC_performance = np.asarray(pickle.load(open('pos_general_neg_bert_eval_metrics-ALC1-bs5.pkl', 'rb')))
RND_performance = np.asarray(pickle.load(open('pos_general_neg_bert_eval_metrics-RND1-bs5.pkl', 'rb')))

RND_marker, ALU_marker, ALC_marker = 'x', '^', 's'
RND_color, ALU_color, ALC_color = 'black', 'red', 'blue'
RND_lstyle, ALU_lstyle, ALC_lstyle = 'dotted', 'solid', 'solid'
RND_fstyle, ALU_fstyle, ALC_fstyle = 'none', 'none', 'none'
RND_msize, ALU_msize, ALC_msize = 5, 5, 5
RND_lwidth, ALU_lwidth, ALC_lwidth = 0.5, 0.5, 0.5

ACH_color = 'black'
ACH_lwidth = 0.5

p_step = float(5)/6274


#### TEST
plt.figure(figsize=(10,6))
#plt.suptitle('Performance of the Test Dataset')
plt.subplot(211)
plt.hlines(y=ACH_performance[0, 4], xmin=0, xmax=len(RND_performance)*(5/6274), label='100% Data',
          color=ACH_color, linewidth=ACH_lwidth) 
plt.plot(np.arange(0, len(RND_performance)*p_step, p_step), ALC_performance[0:, 4], label='AL-Clustering', 
         marker=ALC_marker, color=ALC_color, linestyle=ALC_lstyle, fillstyle=ALC_fstyle, 
         markersize=ALC_msize, linewidth=ALC_lwidth)
plt.plot(np.arange(0, len(RND_performance)*p_step, p_step), ALU_performance[0:, 4], label='AL-Uncertainty', 
         marker=ALU_marker, color=ALU_color, linestyle=ALU_lstyle, fillstyle=ALU_fstyle, 
         markersize=ALU_msize, linewidth=ALC_lwidth)
plt.plot(np.arange(0, len(RND_performance)*p_step, p_step), RND_performance[0:, 4], label='Random', 
         marker=RND_marker, color=RND_color, linestyle=RND_lstyle, fillstyle=RND_fstyle, 
         markersize=RND_msize, linewidth=ALC_lwidth)
plt.legend(loc=4)
#plt.xlabel('Number of active iterations')
plt.xlabel('Percentage of data used')
plt.ylabel('Accuracy')
plt.xlim([0, len(RND_performance)*p_step])

plt.subplot(212)
plt.hlines(y=ACH_performance[0, 5], xmin=0, xmax=len(RND_performance)*(5/6274), label='100% Data',
          color=ACH_color, linewidth=ACH_lwidth) 
plt.plot(np.arange(0, len(RND_performance)*p_step, p_step), ALC_performance[0:, 5], label='AL-Clustering', 
         marker=ALC_marker, color=ALC_color, linestyle=ALC_lstyle, fillstyle=ALC_fstyle, 
         markersize=ALC_msize, linewidth=ALC_lwidth)
plt.plot(np.arange(0, len(RND_performance)*p_step, p_step), ALU_performance[0:, 5], label='AL-Uncertainty', 
         marker=ALU_marker, color=ALU_color, linestyle=ALU_lstyle, fillstyle=ALU_fstyle, 
         markersize=ALU_msize, linewidth=ALC_lwidth)
plt.plot(np.arange(0, len(RND_performance)*p_step, p_step), RND_performance[0:, 5], label='Random', 
         marker=RND_marker, color=RND_color, linestyle=RND_lstyle, fillstyle=RND_fstyle, 
         markersize=RND_msize, linewidth=ALC_lwidth)
plt.legend(loc=4)
#plt.xlabel('Number of active iterations')
plt.xlabel('Percentage of data used')
plt.ylabel('F1 Score')
plt.xlim([0, len(RND_performance)*p_step])

plt.tight_layout(pad=3.0)
%config InlineBackend.figure_format = 'svg'

# plt.savefig('Results-pos_general_neg_bert_xgb-test.svg', format='svg')
plt.show()


TypeError: scalar() argument 1 must be numpy.dtype, not numpy.dtype

'1.15.4'